In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv, jvp, yv
from scipy.optimize import root_scalar
plt.rcParams.update({'font.size': 22})

global kh, cL1, cT1, cL2, cT2, nu

def D_matrix(omega):
        '''
        Determinant of coefficients for the 6x6 matrix in the paper of Christine Valle.
        kh = dimensionless wavenumber (equivalent to kp in Cerv's code)
        b = Outer radius
        a = Inner radius
        cL = Longitudinal velocity of medium 
        cT = Transversal velocity of medium 
        '''    
        #k = omega/c
        #k_hat = k*b
        #omega_m1 = omega*h/cT1
        #k_m = k*h
        h = b - a
        nu = a/b
        omega_h =  omega*h/(cT*(1-nu)) #omega*h/cT 
        kappa = cL/cT
        # The coefficients of the matrix
        # 1st row ################################################################################################################
        d11 = (jv(kh-2,omega_h/kappa) + jv(kh+2,omega_h/kappa) - 2*(kappa**2 -1)*jv(kh,omega_h/kappa))*(omega_h/kappa)**2 # Done
        d12 = (yv(kh-2,omega_h/kappa) + yv(kh+2,omega_h/kappa) - 2*(kappa**2 -1)*yv(kh,omega_h/kappa))*(omega_h/kappa)**2 # Done
        d13 = (jv(kh-2,omega_h) - jv(kh+2,omega_h))*1j*omega_h**2 # Done
        d14 = (yv(kh-2,omega_h) - yv(kh+2,omega_h))*1j*omega_h**2 # Done
        # 2nd row ################################################################################################################
        d21 = (jv(kh-2,nu*omega_h/kappa) + jv(kh+2,nu*omega_h/kappa) - 2*(kappa**2 -1)*jv(kh,nu*omega_h/kappa))*(nu*omega_h/kappa)**2 # Done
        d22 = (yv(kh-2,nu*omega_h/kappa) + yv(kh+2,nu*omega_h/kappa) - 2*(kappa**2 -1)*yv(kh,nu*omega_h/kappa))*(nu*omega_h/kappa)**2 # Done
        d23 = (jv(kh-2,nu*omega_h) - jv(kh+2,nu*omega_h))*1j*(nu*omega_h)**2 # Done
        d24 = (yv(kh-2,nu*omega_h) - yv(kh+2,nu*omega_h))*1j*(nu*omega_h)**2 # Done
        # 3rd row ################################################################################################################
        d31 = (jv(kh-2,omega_h/kappa) - jv(kh+2,omega_h/kappa))*1j*(omega_h/kappa)**2 # Done
        d32 = (yv(kh-2,omega_h/kappa) - yv(kh+2,omega_h/kappa))*1j*(omega_h/kappa)**2 # Done
        d33 = (jv(kh-2,omega_h) + jv(kh+2,omega_h))*-1.0*omega_h**2 # Done
        d34 = (yv(kh-2,omega_h) + yv(kh+2,omega_h))*-1.0*omega_h**2 # Done
        # 4th row ################################################################################################################
        d41 = (jv(kh-2,nu*omega_h/kappa) - jv(kh+2,nu*omega_h/kappa))*1j*(nu*omega_h/kappa)**2 # Done
        d42 = (yv(kh-2,nu*omega_h/kappa) - yv(kh+2,nu*omega_h/kappa))*1j*(nu*omega_h/kappa)**2 # Done
        d43 = (jv(kh-2,nu*omega_h) + jv(kh+2,nu*omega_h))*-1.0*(nu*omega_h)**2 # Done
        d44 = (yv(kh-2,nu*omega_h) + yv(kh+2,nu*omega_h))*-1.0*(nu*omega_h)**2 # Done
        # The dij matrix #########################################################################################################
        dij = np.array([
                        [d11 ,d12, d13, d14], 
                        [d21, d22, d23, d24],
                        [d31, d32, d33, d34],
                        [d41, d42, d43, d44],                        
                       ])
        return dij

def D_det(omega): 
    '''Calculate the determinant of matrix '''
    return np.linalg.det(D_matrix(omega))

def VpVs(poisson):
    return np.sqrt((2*(1-poisson))/(1-2*poisson))

# DISPERSION OF ELASTIC WAVES AND RAYLEIGH-TYPE WAVES IN A ANNULUS.

This notebook summarizes the calculations presented in the research article:

- Liu, Guoli, and Jianmin Qu. "Guided circumferential waves in a circular annulus." (1998): 424-430.
  
In this notebook, for a practical implementation, we use a $V_P = 5660~m/s$ which corresponds to Aluminium.  The radius of the disc is $2.5~cm$ ($25~mm$).

In [2]:
# CONSTANTS
r1 = 0.0628 #0.0127#0.02225
nu = 0.1 # eta in paper, sorry for change of notation!.  Inner-to-outer radius proportion 
r2 = nu*r1
r = r1
C = 2*np.pi*r1
cL1 = 5660 # m/s
cT1 = 3120 # m/s
cR = 2932.9
f = 1.0*1e6 # Hz
omega = 2*np.pi*f
wl = cL1/f
c = omega*wl/(2*np.pi)
kp = 2*np.pi*r1/wl # kappa (angular wavenumber - dimensionless)
phi = cT1/cL1

print('Circumference %3.2f mm' % (C*1000))
print('wavelength: %3.2f mm' % (wl*1000))
print('Speed (c):  %3.2f m/s' %  c)
print('Angular wavenumber (kp): %3.2f' % kp)
print('Vs/Vp (phi):  %3.2f' % phi)

Circumference 394.58 mm
wavelength: 5.66 mm
Speed (c):  5660.00 m/s
Angular wavenumber (kp): 69.71
Vs/Vp (phi):  0.55


## Re-scaling of values

By dividing the velocities by a $1000$ and multiplying the radius by the same factor, the velocity units are expressed in $mm/\mu s$ and the radius in $mm$.

In [3]:
# Re-scale the values
cS1 = cT1/1000
cP1 = cL1/1000
rb = 1000*r1 
ra = 1000*r2
nu=ra/rb
#radius = 1000*r1

## Solve for the roots of the determinant 

The system of equations can be written as a 6x6 matrix.  The condition for non-trivial solutions requires: 

$$det(D(k,w))$$

For each $k$ value along all the frequencies considered, the function $det(D(k,w))$ cancels out for a set of values of $w$ which are the roots of the characteristic equation of the matrix. 

The roots are found by the bisection method.

In [4]:
%%time
# Define constants and range of calculations
fstop = 5.0 # Ending frequency in MHz
wstart = 0.0001 #0.0001  
dw = 0.01 #0.001#0.001; #0.01
kpmax = 12 # 
kpstart = 0.01 #
kpstop = kpmax/(1-nu) # (if nu = 0.5, kstop = 24) (if nu = 0.1, kstop )

omegas = np.arange(wstart,2*np.pi*fstop,dw)
kps = np.linspace(kpstart,kpstop,100) #np.arange(kpstart,kpstop,dp)
dp = np.diff(kps).mean()

a = ra
b = rb
h = b - a
cT = cS1
cL = cP1
arguments = (a, b, cT, cL)

# Calculate and save the roots using bisection method
number_of_roots = 10 

roots = []
KP = []
for kh in kps: #kp 
    print(kh)    
    rc = 0        
    for omega in omegas: 
        if kh <= 1:
            NR = number_of_roots  #number_of_roots = NR-1
            if rc < NR: #number_of_roots:
                if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                    #print(omega)\
                    root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                    if root_val != None:
                        rc = rc + 1
                        roots.append(root_val.root)
                        KP.append(kh)
                        print(kh,root_val.root)                        
            elif rc == NR:
                break
        elif kh > 1:            
            NR = number_of_roots#number_of_roots = NR
            if rc < NR: #number_of_roots:
                if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                    #print(omega)\
                    root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root                    
                    if root_val != None:
                        rc = rc + 1
                        roots.append(root_val.root)
                        KP.append(kh)
                        print(kh,root_val.root)
                        if rc == 1:
                            wstart = root_val.root#-3*dw
                            omegas = np.arange(wstart,2*np.pi*fstop,dw)
            elif rc == NR:
                    break

0.01
0.01 0.0005163360595703125
0.01 0.1815289093017578
0.01 0.2554995513916016
0.01 0.4201122833251953
0.01 0.4468633819580079
0.01 0.5831895233154297
0.01 0.7093321014404297
0.01 0.7474340606689452
0.01 0.9133154083251954
0.01 0.995061471557617
0.14457912457912458
0.14457912457912458 0.007344491577148438
0.14457912457912458 0.17790616760253908
0.14457912457912458 0.25941465148925785
0.14457912457912458 0.4103071380615235
0.14457912457912458 0.45618009338378906
0.14457912457912458 0.5833666778564452
0.14457912457912458 0.707052896118164
0.14457912457912458 0.7515884185791015
0.14457912457912458 0.9134547210693359
0.14457912457912458 0.9954159332275391
0.27915824915824916
0.27915824915824916 0.013515145874023438
0.27915824915824916 0.16967435607910153
0.27915824915824916 0.26859739074707034
0.27915824915824916 0.3947649932861329
0.27915824915824916 0.4703057647705079
0.27915824915824916 0.584111001586914
0.27915824915824916 0.7026183868408205
0.27915824915824916 0.7609684539794922
0.27

/home/leo/anaconda3/envs/jrudge/lib/python3.10/site-packages/scipy/optimize/_zeros_py.py:570: ComplexWarning: Casting complex values to real discards the imaginary part
  r = _zeros._bisect(f, a, b, xtol, rtol, maxiter, args, full_output, disp)


0.27915824915824916 0.9963393951416015
0.41373737373737374
0.41373737373737374 0.018284127807617186
0.41373737373737374 0.15960279235839842
0.41373737373737374 0.28072492370605473
0.41373737373737374 0.378914926147461
0.41373737373737374 0.48360379943847653
0.41373737373737374 0.5860964752197264
0.41373737373737374 0.698082406616211
0.41373737373737374 0.7730138946533204
0.41373737373737374 0.914963510131836
0.41373737373737374 0.997721078491211
0.5483164983164983
0.5483164983164983 0.02068753967285156
0.5483164983164983 0.14995054016113282
0.5483164983164983 0.29431028137207027
0.5483164983164983 0.3640951324462891
0.5483164983164983 0.4945132232666016
0.5483164983164983 0.590214974975586
0.5483164983164983 0.6942674041748048
0.5483164983164983 0.7861726165771485
0.5483164983164983 0.9169161773681641
0.5483164983164983 0.9994372344970703
0.6828956228956229
0.6828956228956229 0.019607980346679686
0.6828956228956229 0.14253843078613282
0.6828956228956229 0.3084814239501953
0.68289562289

In [5]:
%matplotlib qt
plt.figure()
plt.scatter(KP,roots)

In [6]:
roots = np.array(roots)
KPvals = np.unique(KP)
if number_of_roots > 1:    
    ROOTS = roots.reshape((len(KPvals),number_of_roots))
else:
    ROOTS = roots
    

# Plot the dispersion curves 

In [7]:
k_ = (1-nu)*KPvals
w_ = ROOTS*h/cT

In [8]:
plt.figure()
plt.plot(k_,w_[:,0], label = 'mode 1')
plt.plot(k_,w_[:,1], label = 'mode 2')
plt.plot(k_,w_[:,2], label = 'mode 3')
plt.plot(k_,w_[:,3], label = 'mode 4')
plt.plot(k_,w_[:,4], label = 'mode 5')
plt.plot(k_,w_[:,5], label = 'mode 6')
plt.plot(k_,w_[:,6], label = 'mode 7')
plt.plot(k_,w_[:,7], label = 'mode 8')
plt.plot(k_,w_[:,8], label = 'mode 9')
plt.plot(k_,w_[:,9], label = 'mode 10')
plt.axis([0.0,12.0,0,25.0])
plt.xlabel(r'$\bar{k}$')
plt.ylabel(r'$\bar{\omega}$')
plt.legend()
plt.grid(True)

In [9]:
plt.figure()
plt.plot(k_,w_[:,0]/k_, label = 'mode 1')
plt.plot(k_,w_[:,1]/k_, label = 'mode 2')
plt.plot(k_,w_[:,2]/k_, label = 'mode 3')
plt.plot(k_,w_[:,3]/k_, label = 'mode 4')
plt.plot(k_,w_[:,4]/k_, label = 'mode 5')
plt.plot(k_,w_[:,5]/k_, label = 'mode 6')
plt.plot(k_,w_[:,6]/k_, label = 'mode 7')
plt.plot(k_,w_[:,7]/k_, label = 'mode 8')
plt.plot(k_,w_[:,8]/k_, label = 'mode 9')
plt.plot(k_,w_[:,9]/k_, label = 'mode 10')
plt.axis([0.0,np.max(k_),0, 4.0])
plt.xlabel(r'$\bar{k}$')
plt.ylabel('c/cT')
plt.grid(True)
plt.legend()

In [10]:
mode1 = w_[:,0]/k_
mode2 = w_[:,1]/k_
mode3 = w_[:,2]/k_
mode4 = w_[:,3]/k_
mode5 = w_[:,4]/k_
mode6 = w_[:,5]/k_
mode7 = w_[:,6]/k_
mode8 = w_[:,7]/k_
mode9 = w_[:,8]/k_
mode10 = w_[:,9]/k_

# Matrix = np.c_[k_, mode1, mode2, mode3, mode4, mode5, mode6, mode7, mode8, mode9, mode10 ]
# np.savetxt('Aluminium_Dispersion_Liu_'+str(np.round(nu,decimals=2))+'_Vel_Rel.mod', Matrix, delimiter = ',')

In [11]:
if number_of_roots ==1:
    mode1 = 1000*w_[:,0]*cT/k_
    plt.figure()
    plt.title('Velocity dispersion')
    plt.plot(k_, mode1, label = 'mode 1')
    plt.xlabel('kp')
    plt.ylabel('c (m/s)')
    plt.axis([0.0,np.max(k_),0, 8000.0])
    plt.legend()
    plt.grid(True)
else:
    mode1 = 1000*w_[:,0]*cT/k_
    mode2 = 1000*w_[:,1]*cT/k_
    mode3 = 1000*w_[:,2]*cT/k_
    mode4 = 1000*w_[:,3]*cT/k_
    mode5 = 1000*w_[:,4]*cT/k_
    mode6 = 1000*w_[:,5]*cT/k_

    plt.figure()
    plt.title('Velocity dispersion')
    plt.plot(k_, mode1, label = 'mode 1')
    plt.plot(k_, mode2, label = 'mode 2')
    plt.plot(k_, mode3, label = 'mode 3')
    plt.plot(k_, mode4, label = 'mode 4')
    plt.plot(k_, mode5, label = 'mode 5')
    plt.plot(k_, mode6, label = 'mode 6')
    plt.axhline(0.9214,lw=0.5,ls='--',c='r')
    plt.xlabel(r'$\bar{k}$')
    plt.ylabel('c (m/s)')
    plt.axis([0.0,np.max(k_),0, 8000.0])
    plt.legend()
    plt.grid(True)

In [12]:
# Matrix = np.c_[k_, w_]
# np.savetxt('Steel_Dispersion_Liu_'+str(np.round(nu,decimals=2))+'.mod', Matrix, delimiter = ',')